In [ ]:
!pip install -U qwen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.7/522.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
import json
from pathlib import Path
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
DEV_DIR = Path("/content/drive/MyDrive/dataset/dev.jsonl")

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os

base_model = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model)


# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype=torch.float16,
)

# 加載你訓練好的 LoRA adapter

model = PeftModel.from_pretrained(
    model,
    "/content/drive/MyDrive/rag_lora_adapter_chatmode"

)

model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [ ]:
def generate_answer(history, current, context):
    contexts_sorted = sorted(context, key=lambda x: x[1], reverse=True)
    formatted = []
    for i, (doc, score) in enumerate(contexts_sorted, start=1):
        formatted.append(
            f"[Document #{i} {doc}]\n"
        )
    context_with_score = "\n".join(formatted)
    if history:
        limited = history[-2:]
        history_text = "\n".join(f"{h[0]}: {h[1]}" for h in limited)
    else:
        history_text = ""

    instruct = f"""
                You are a RAG answer generator.
                Use the reference documents as the main source of information to answer the question.
                If the reference contains partial clues, synthesize them to answer the question.
                Answer concisely (1–2 sentences).
                If the reference is empty, answer: I don't know.
                Do not reply with “yes” or “no” alone. Always provide a short explanation based on the context.
    """.strip()
    prompt = f"""
CONTEXT:
{context_with_score}

HISTORY:
{history_text}

QUESTION:
{current}

Answer:
            """.strip()
    messages = [
      {"role": "system", "content": instruct},
      {"role": "user", "content": prompt}
    ]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    output_ids = model.generate(
          **inputs,
          max_new_tokens=256,
          do_sample=False,
          temperature=0.0,
          eos_token_id=tokenizer.eos_token_id,
      )

    generated = output_ids[0]
    answer_ids = generated[len(inputs["input_ids"][0]):]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    #fin_answer = json.loads(json_answer)
    #answer = fin_answer["text"]
    return answer.strip()


In [ ]:
history =  [["user", "What are the steps involved to delete the CIS instance?"], ["agent", "To delete a CIS instance, you must first delete all global load balancers, pools, and health checks. Then, delete the associated domain (zone). Go to the Overview page and click the trash can icon next to the domain name located in the Service Details section to start the deletion process.\nIf you are moving your domain to a different provider, be sure to migrate your DNS records and other configuration information to the new provider before activating the domain there. Activating the domain before migrating from CIS can cause your domain to change to a \"Moved\" state.\nYou can visit the IBM Cloud Internet Services for more information on how to delete the CIS instance. Delete the CIS instance if you created it specifically for this tutorial.\n\nRelated content\n\nIBM Cloud Internet Service"], ["user", "Do I need to import or recreate my DNS records in CIS?"], ["agent", "No, before you switch the traffic for your domain to CIS, it's important to import or recreate your DNS records in CIS. Although you can skip this step, it's crucial to ensure that your DNS records are correctly configured in CIS to avoid any parts of your website becoming inaccessible. Here are the command options: DNS_DOMAIN_ID: The ID of the DNS domain. Since CIS doesn't support email traffic by default, you need to set the PTR record to the location of your email server. You may need to reach out to your email provider for help. Import records - By default, a total of 3500 DNS records are allowed (imported and created on CIS). "], ["user", "Can I add a custom domain?"], ["agent", "To add a custom domain to IBM Cloud Internet Services, follow these steps:\n1. Go to the Overview page of your CIS instance.\n2. Under the Domain section, click the plus (+) icon next to your domain.\n3. choose the 'Connect domain' option in the revealed section.\n4. Complete the process by setting up DNS records and delegating domain management as instructed in the provided document.\nAfter adding a new domain, you can switch between your domains using the list menu in the Domain section to configure settings for each domain.\nWork with your Domain Name Service (DNS) provider or IBM Cloud DNS to register your custom domain. If the apps you want Ingress to expose are in different namespaces in one cluster, register the custom domain as a wildcard domain, such as .custom_domain.net. Note that custom domains are required to use Ingress with private ALBs."], ["user", "What is the transportation layer?"], ["agent", "Mutual Transport Layer Security (mTLS) authentication ensures that traffic is secure and trusted in both directions between a client and server. This feature is exclusively available for customers at the Enterprise or Security plan level. \nWhen mTLS is configured, access is only granted to requests with a corresponding client certificate. Upon reaching the application, CIS requests the client certificate. If the client fails to present the certificate, the request is denied. Otherwise, the key exchange proceeds.\nThe Transport Layer Security (TLS) options allow you to control whether visitors can browse your website over a secure connection and, if they do, how IBM Cloud\u00ae Internet Services connects to your origin server. Mutual TLS is not enabled by default. It is an additional service that requires prior authorization and enablement."], ["user", "Is privacy protected?"], ["agent", "Yes. Data privacy is upheld through the use of private data collections. For example, multiple wholesalers and a group of farmers can be connected to a single channel. If a farmer and a wholesaler want to conduct a private transaction, they can create a channel for this purpose. Alternatively, they can choose to establish a private data collection on the smart contract governing their business interactions to maintain privacy regarding sensitive aspects of the sale, such as the price, without the need to create an additional channel. This policy applies to EU-US and Swiss-US customers. If a government wants access to data held by IBM on behalf of a SaaS client, IBM expects that government to deal directly with that client. Data privacy and governance are of utmost importance. As a leader in providing a fully managed and globally distributable Database-as-a-Service, IBM\u00ae Cloudant\u00ae for IBM Cloud\u00ae allows customers to position data in any global IBM Cloud\u00ae or AWS region. By providing customers with high levels of data mobility to meet their local needs, IBM\u00ae and IBM Cloudant take data privacy and governance seriously. IBM Cloud Delivery Services does not use DLP monitoring. Access controls are implemented on all databases restricted to privileged users only."]]
current = "What about security?"
context =[("Setting Transport Layer Security (TLS) options \n\nThe Transport Layer Security (TLS) options let you control whether visitors can browse your website over a secure connection, and when they do, how IBM Cloud\u00ae Internet Services connects to your origin server.\n\nUse the latest version of the TLS protocol (TLS 1.3) for improved security and performance by switching from Off to On.\n\n\n\n TLS encryption modes \n\nSet the TLS mode by selecting one of the following options from the Mode list.\n\nThese options are listed in the order from the least secure (Off) to the most secure (End-to-End CA signed).\n\n\n\n* [Off](https://cloud.ibm.com/docs/cis?topic=cis-cis-tls-optionstls-encryption-modes-off) (not recommended)\n* [Client-to-Edge](https://cloud.ibm.com/docs/cis?topic=cis-cis-tls-optionstls-encryption-modes-client-to-edge) (edge to origin not encrypted, self-signed certificates are not supported)\n* [End-to-End flexible](https://cloud.ibm.com/docs/cis?topic=cis-cis-tls-optionstls-encryption-modes-end-to-end-flexible) (edge to origin certificates can be self-signed)\n* [End-to-End CA signed](https://cloud.ibm.com/docs/cis?topic=cis-cis-tls-optionstls-encryption-modes-end-to-end-ca-signed) (default and recommended)\n* [HTTPS only origin pull](https://cloud.ibm.com/docs/cis?topic=cis-cis-tls-optionstls-encryption-modes-origin-only-pull) (Enterprise only)\n\n\n\n\n\n Off \n\nNo secure connection between your visitor and CIS, and no secure connection between CIS and your web server. Visitors can only view your website over HTTP, and any visitor attempting to connect using HTTPS receives an HTTP 301 Redirect to the plain HTTP version of your website.\n\nZoom\n\n!", 12.575332), ("\nUsing mutual TLS \n\nMutual Transport Layer Security (mTLS) authentication ensures that traffic is both secure and trusted in both directions between a client and server. It is only available for customers at the Enterprise or Security plan level.\n\nWhen mTLS is configured, access is granted only to requests with a corresponding client certificate. When a request reaches the application, CIS responds with a request for the client certificate. If the client fails to present the certificate, the request is not allowed to proceed. Otherwise, the key exchange proceeds.\n\nZoom\n\n![Diagram of mTLS handshake](https://cloud.ibm.com/docs-content/v1/content/cb1eb27836421578019401fa7556779109430b29/cis/images/mtls-handshake.png)\n\nFigure 1. Diagram of an mTLS handshake\n\n\n\n Configuring mutual TLS \n\nMutual TLS is not enabled by default. It is an additional service that requires prior authorization and enablement.\n\nTo obtain authorization, you must submit an IBM Support case.\n\nAfter mTLS is turned on for your account, take the following steps to enable it.\n\n\n\n1. Navigate to the Security page in the CIS UI.\n2. Select the Mutual TLS tab.\n3. Click Enable to enable the feature.\n\n\n\nAfter mTLS is enabled, it cannot be disabled.\n\nTo set up mTLS authentication in the IBM Cloud Internet Services UI for a particular endpoint:\n\n\n\n1. In the Root certificates table, click Add to define a new root certificate.\n2. Paste the certificate content into the content field, provide a name for the Root CA, and add one or more fully qualified domain names (FQDN) of the endpoints that you want to use this certificate. These FQDNs are the hostnames that are used for the resources being protected by the application policy. You must associate the Root CA with the FQDN that the application being protected uses.\n3. Click Save.\n\nIf your zone is using an intermediate certificate in addition to the root certificate, upload the entire chain.\n4.", 15.929632), ("\nA Transport Layer Security (TLS) secret contains a signed TLS certificate, including all its intermediate certificates, and its corresponding private key from a certificate authority (CA). Use TLS secrets when you work with custom domain mappings.\n\nTo see CLI help for creating a TLS secret, run ibmcloud ce secret create --format tls.\n\nThe following example creates an TLS secret that is named mysecret-tls. The certificate chain that corresponds to the custom domain is contained in the file certificate.txt and the matching private key file is contained in the file privatekey.txt. Both of these files are located in the root directory of the local workstation.\n\nibmcloud ce secret create --name mysecret-tls --format tls --cert-chain-file certificate.txt --private-key-file privatekey.txt\n\n\n\n\n\n Example output for a TLS secret \n\nCreating TLS secret mysecret-tls...\nOK\n\n\n\n\n\n\n\n ibmcloud ce secret delete \n\nDelete a secret.\n\nibmcloud ce secret delete --name SECRET_NAME [--force] [--ignore-not-found] [--quiet]\n\n\n\n Command Options \n\n--name, -n\n: The name of the secret. This value is required.\n\n--force, -f\n: Force deletion without confirmation. This value is optional. The default value is false.\n\n--ignore-not-found, --inf\n: If not found, do not fail. This value is optional. The default value is false.\n\n--quiet, -q\n: Specify this option to reduce the output of the command. This value is optional. The default value is false.\n\n\n\n\n\n Example \n\nibmcloud ce secret delete --name mysecret-genericfromfile -f\n\n\n\n\n\n Example output \n\nDeleting secret mysecret-genericfromfile...\nOK\n\n\n\n\n\n\n\n ibmcloud ce secret get \n\nDisplay the details of a secret.", 14.570643)]
generate_answer(history, current, context)

'Security is ensured through mechanisms like mutual TLS (mTLS) authentication, which provides secure and trusted communication in both directions between clients and servers. Additionally, TLS options can be configured to enhance security, with the default and recommended setting being End-to-End CA signed, which ensures secure connections between the edge and the origin server.'

In [ ]:
output_path = "/content/drive/MyDrive/dataset/dev_with_pred_chatmode.jsonl"
with open(DEV_DIR, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)


        history = [(cov["speaker"],cov["text"]) for cov in obj["input"]][:-1]
        current = [x["text"] for x in obj["input"]][-1]
        context = [(c["text"], c["score"]) for c in obj["contexts"]]

        # generator
        prediction = generate_answer(history, current, context)

        obj["predictions"] = [{"text": prediction}]

        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")